In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Defining the network

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # conv layers
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        
        # fc layers
        self.fc1 = nn.Linear(in_features=16*5*5, out_features=120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a 2x2 window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)     
        

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
params_learnable = list(net.parameters())
print(len(params_learnable))
print(params_learnable[0].size()) # conv1's weight

10
torch.Size([6, 1, 5, 5])


In [6]:
input = torch.randn(1, 1, 32, 32) # input images will be of size 32x32
out = net(input)
print(out)

tensor([[-0.0279,  0.0555, -0.1139, -0.1128, -0.0451, -0.0589,  0.0624,  0.0042,
         -0.1324, -0.0167]], grad_fn=<AddmmBackward>)


In [7]:
net.zero_grad()
out.backward(torch.randn(1,10))

### Loss Function

In [15]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1) #reshaping the target to output shape
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.5660, grad_fn=<MseLossBackward>)


In [16]:
print(loss.grad_fn) #MSE Loss
print(loss.grad_fn.next_functions[0][0]) #Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) #ReLU

### Backpropagation

In [17]:
net.zero_grad()

print('conv1.bias.grad before backward propagation')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward propagation')
print(net.conv1.bias.grad)

conv1.bias.grad before backward propagation
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward propagation
tensor([ 0.0049, -0.0155,  0.0024,  0.0104,  0.0051, -0.0125])


In [18]:
learning_rate = 0.01

for f in net.parameters():
    f.data.sub_(f.grad.data*learning_rate)

different update rules already have <b>built in optimizers</b>

In [19]:
import torch.optim as optim

# creation of an optimizer
optimizer = optim.SGD(net.parameters(), lr = 0.01)

# in our training loop:
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() #updates weights